# Emission Models

To simplify the construction of complex spectra with many contributing components and different operations involved in their construction Synthesizer uses `EmissionModels`. These objects define the generation of a single spectra in the construction of a more complex whole. The possible operations that an `EmissionModel` can define are:
- Extraction of a spectra from a `Grid` (see the `Grid` [docs](grids/grids_example.ipynb)).
- Combination of different spectra.
- Generation of dust emission spectra (see the dust emission [docs](misc/dust_emission.ipynb)).
- Attenuation of a spectra with a dust curve (see the dust attenuation docs [WIP]()).

In addition any one of these operations can also be done in the presence of a mask to apply the operation to a subset of the components contents (either particles or a parametric model).

Once an `EmissionModel` is constructed it can be passed to any `get_spectra` method on an emitter (a `Galaxy` or galaxy component) to get the spectra defined in the passed `EmissionModel` for that emitter. For more details see [generating a parametric spectra](parametric/generate_sed.ipynb) or generating a particle spectra [WIP]().

## Emission model dependencies

Each of operation a model can define depends on different properties being set.

For extraction operations you need:
- A grid to extract from (`grid`).
- A spectra key to extract (`extract`).
- Optionally, an escape fraction to apply which defaults to 0.0 (`fesc`).

For combination operations you need:
- The a list of models which will be combined into the resultant spectra (`combine`).

For generation of dust emission you need:
- A [dust emission model](misc/dust_emission.ipynb) from which to generate spectra (`dust_emission_model`).

For attenuation operations you need:
- The [dust curve]() to apply (`dust_curve`).
- The model to apply the attenuation to (`apply_dust_to`).
- The optical depth to use with the dust curve (`tau_v`).

As mentioned, masking can be applied alongside any of these operations. Additionally, any number of masks can be combined on the same operation. Each mask is defined by:
- The attribute of the component to mask on (`mask_attr`).
- The threshold of the mask (`thresh`).
- The operator to use when generating the mask, i.e. "<", ">", "<=", ">=", "==", or "!=" (`op`).

We'll cover all of these operations and parameters in more detail below.

## Premade Models

In the sections below we'll cover setting up your own model but most use cases will want a common set of spectra with a common set of properties. To avoid unnecessarily redefining common models every time Synthesizer is used we provide some premade models which can be used "out of the box" or as the foundation for constructing more complex models.

These premade models can be imported directly from the `emission_models` submodule which also defines a list containing all available models we have defined.

In [ ]:
from synthesizer.emission_models import PREMADE_MODELS

print(PREMADE_MODELS)

To use any of these models the model simply needs importing and the required arguments passed, which arguments are required depends on the exact operation, more complex models further down the model "tree" will require more parameters to pass to the models which make them up.

Here is a demonstration of initialising different premade models with differing levels of complexity.

In [ ]:
from synthesizer.dust.attenuation import PowerLaw
from synthesizer.dust.emission import Blackbody
from synthesizer.emission_models import (
    EmergentEmission,
    TotalEmission,
    TransmittedEmission,
)
from synthesizer.grid import Grid

# Get the grid which we'll need for extraction
grid_dir = "../../tests/test_grid"
grid_name = "test_grid"
grid = Grid(grid_name, grid_dir=grid_dir)

# Get a simple extraction model (with optional fesc)
transmitted = TransmittedEmission(grid=grid, fesc=0.3)

# Get the emergent model which combines the escaped
# emission with an attenuated emission (here using incident
# for demonstration purposes) and requires some dust
# attenuation properties
emergent = EmergentEmission(
    grid=grid,
    dust_curve=PowerLaw(slope=-1),
    apply_dust_to=transmitted,
    tau_v=0.67,
    fesc=0.2,
)

# Finally, define a Total emission model which
# is the combination of all other spectra
model = TotalEmission(
    grid=grid,
    dust_curve=PowerLaw(slope=-1),
    tau_v=0.67,
    dust_emission_model=Blackbody,
)

With these model now defined they can be passed into any emitter's spectra generation method but you can also visualise the different spectra and operations encapsulated by a model and see the "model tree". To do see we can use the `plot_emission_model` method.

Lets take a look at the total emission model.

In [ ]:
# Plot the model tree
model.plot_emission_tree()

Here we can see each individual spectra generated by the model and how they relate. Solid lines denote combinations of spectra into another, dashed lines denote an attenuation of the child spectra to produce the parent, blue boxes have no mask applied, green boxes (which we currently have none of) have a mask applied to them, rounded boxes are the result of a non-extraction operation, and square boxes are an extraction operation. You'll notice the extraction operations are always leaves in the tree.

Note that omitting `dust_emission_model` when initialising `TotalEmission` will result in a simpler model where `total == emergent`.

### Charlot and Fall

In addition to the simpler models shown above we also implement some more complex models. The first of these follows Charlot&Fall+2000:

- ...



### Pacman

We also implement a more generalised form of Charlot&Fall+2000 which we call "Pacman":

- ...


## Modifying a model

Of course, a premade model may not behave in the exact way you want for your application but it may not be different enough to warrant making your own from scratch. In these instances any properties of a model can be modified or changed. To do so we simply call a method which can change or add to a property on either the base model (`label=None`), a specific model (e.g. `label="nebular"`), or (for certain properties) all applicable models (`set_all=True`).

You can add a mask.

In [ ]:
from unyt import dimensionless

model.add_mask("log10ages", 7 * dimensionless, "<", label="nebular_continuum")
model.add_mask("log10ages", 7 * dimensionless, "<", label="linecont")

Here we've just added a filter for only young stars for the nebular component. Notice however, that we have had to supply units alongside our threshold, this is required to maintain consistency across objects but also enables arbitrary units to be used for a threshold.

We can see the effect of this in the emission model tree.

In [ ]:
model.plot_emission_tree()



You can change the grid (here we'll just pass the same grid again for demonstration purposes).

In [ ]:
model.set_grid(grid, set_all=True)

Notice that we've changed the grid for the whole model by passing `set_all=True`.

We can also change dust properties but for these we don't have to do each individually and can instead call a single method.

In [ ]:
model.set_dust_props(
    dust_curve=PowerLaw(slope=-0.7),
    apply_dust_to=model["reprocessed"],
    tau_v=0.7,
    label="attenuated",
)

Notice we've used `model["reprocessed"]` to access the reprocessed model that's already part of the `EmissionModel` in the above, an `EmissionModel` supports dicitonary like key indexing to extract models by label.

As well as just changing the properties of existing models we can also replace a model with one or more models. As an example, below we will swap out the attenuation model with two separate attenuation models which apply different dust curves to the old and young population (for further information on defining your own models see below).

In [ ]:
from synthesizer.emission_models import EmissionModel

# Define the models well replace attenuate with
young_attenuated = EmissionModel(
    "young_attenuated",
    dust_curve=PowerLaw(slope=-1),
    apply_dust_to=model["reprocessed"],
    tau_v=0.7,
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<=",
)
old_attenuated = EmissionModel(
    "old_attenuated",
    dust_curve=PowerLaw(slope=-1),
    apply_dust_to=model["reprocessed"],
    tau_v=0.7 * 0.67,
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op=">",
)

model.replace_model("attenuated", young_attenuated, old_attenuated)
model.plot_emission_tree()

Although not shown here explicitly, passing a single model for the replacement will just swap out the model. 

## Creating your own EmissionModel

To create your own `EmissionModel` all you need to do is define each indiviudal spectra and chain them together. 

### Defining an extraction

To define an extraction we simply need to pass a `Grid` to extract from and a spectra key to extract (with the option of providing an escape fraction).

In [ ]:
transmitted = EmissionModel(
    "transmitted", grid=grid, extract="transmitted", fesc=0.3
)

You'll notice this is similar to using the premade models detailed above but now we have freedom over the `label` too (the first argument).

### Defining a combination

To define a combination we simply pass the models we want combined to the `combine` keyword (along with a label).

In [ ]:
# Define models to combine
linecont = EmissionModel(
    "linecont",
    grid=grid,
    extract="linecont",
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<",
    fesc=0.7,
)
nebular_cont = EmissionModel(
    "nebular_continuum",
    grid=grid,
    extract="nebular_continuum",
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<",
)

# Define the combined model
nebular = EmissionModel("nebular", combine=(linecont, nebular_cont))

### Defining an attenuation

To define an attenuated spectra we need a dust curve, the spectra to apply the dust to, and an optical depth (once again along with a label).

One thing we haven't mentioned up until now is that `tau_v` and `fesc` can both have strings passed instead of numbers. When a string is passed the spectra generator method will extract the attribute of a component stated in the string and use those extracted values for `tau_v` or `fesc`.

In [ ]:
attenuated = EmissionModel(
    "attenuated",
    dust_curve=PowerLaw(slope=-1),
    apply_dust_to=nebular,
    tau_v="tau_v",
)

### Including a mask

A mask can be included in any step by passing `mask_attr`, `mask_thresh`, and `mask_op`.

In [ ]:
masked_transmitted = EmissionModel(
    "masked_transmitted",
    grid=grid,
    extract="transmitted",
    fesc="fesc",
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<",
)

But we aren't tied to having only a single mask on a step. If we want to add more masks we can use `add_mask` as we did on the premade models, these will be combined with an `and` at the point of spectra generation.

In [ ]:
masked_transmitted.add_mask("metallicities", 0.01 * dimensionless, "<")
masked_transmitted.add_mask("log10ages", 6 * dimensionless, ">")

In the code below we'll reconstruct the `TotalEmission` model with the Charlot&Fall+2000 like attenuation operation explicitly to demonstrate constructing a complex model.

In [ ]:
# Define the extractions
transmitted = EmissionModel(
    "transmitted",
    grid=grid,
    extract="transmitted",
    fesc=0.3,
)
escaped = EmissionModel("escaped", grid=grid, extract="transmitted", fesc=0.7)
linecont = EmissionModel(
    "linecont",
    grid=grid,
    extract="linecont",
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<",
    fesc=0.7,
)
nebular_cont = EmissionModel(
    "nebular_continuum",
    grid=grid,
    extract="nebular_continuum",
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<",
)

# Combine the extractions
nebular = EmissionModel("nebular", combine=(linecont, nebular_cont))
reprocessed = EmissionModel("reprocessed", combine=(nebular, transmitted))

# Apply the young and old dust attenuation
young_attenuated = EmissionModel(
    "young_attenuated",
    dust_curve=PowerLaw,
    apply_dust_to=reprocessed,
    tau_v=("tau_v", 0.67),
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<=",
)
old_attenuated = EmissionModel(
    "old_attenuated",
    dust_curve=PowerLaw,
    apply_dust_to=reprocessed,
    tau_v="tau_v",
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op=">",
)

# And combine them into a single attenuated spectra
attenuated = EmissionModel(
    "attenuated", combine=(young_attenuated, old_attenuated)
)
emergent = EmissionModel("emergent", combine=(attenuated, escaped))

# Create a dust emission model
dust_emission = EmissionModel("dust_emission", dust_emission_model=Blackbody)

# And bring everything together into the total emission
total = EmissionModel("total", combine=(emergent, dust_emission))

total.plot_emission_tree()